# Feature Extraction

The dataset are available for [download here](https://javerianacaliedu-my.sharepoint.com/:f:/g/personal/ccdelgado_javerianacali_edu_co/Estk6ZSR0CpEkqADhvUfnHABsrVhkDKPi19S47bdG4T4sg?e=Yo5dQr).

In [10]:
# Import libraries
import os
import glob
import numpy as np
import pandas as pd
import geopandas as gpd
import pipeline as pipe

from tqdm import tqdm

In [11]:
#Directory
inDir = r'C:\Users\crist\OneDrive - Pontificia Universidad Javeriana Cali\MDPI'           
os.chdir(inDir)

In [12]:
#Files
tifFiles = glob.glob('data\**\orthomosaics\**.tif')
CPFiles = glob.glob('data\**\shapes\**_CP.shp')
plotFiles = glob.glob('data\**\shapes\**_PLOTS.shp')

In [14]:
#Do the job!
NDSI_stats = pd.DataFrame()
NDWI_stats = pd.DataFrame()
 
for i, imgDir in tqdm(enumerate(tifFiles)):
    season = imgDir.split("\\")[1]
    dataset = imgDir.split("\\")[3].split(".")[0]
    print('Processing dataset:' + dataset)
    
    #Read image and metadata
    img, x_sz, y_sz, n_band, ext, proj, datatype = pipe.readImg(imgDir)
    
    #Rasterize Panels
    Panels_ds = pipe.rasterize(CPFiles[i], y_sz, x_sz, ext, proj, ['ALL_TOUCHED=FALSE', 'ATTRIBUTE=ID'])
    roiPanels = Panels_ds.GetRasterBand(1).ReadAsArray().astype(np.uint32)
    
    #ELC
    img_elc = pipe.ELC(img, roiPanels, y_sz, x_sz, n_band, datatype)
    
    #Crop masking
    Field_ds = pipe.rasterize(plotFiles[i], y_sz, x_sz, ext, proj, ['ALL_TOUCHED=FALSE', 'ATTRIBUTE=PLOT'])
    roiField = Field_ds.GetRasterBand(1).ReadAsArray().astype(np.uint32)
    
    img_field = pipe.createEmpty(y_sz, x_sz, n_band, datatype)
    for band in range(n_band):
        img_field[:, :, band] = img[:, :, band]*roiField
        
    CC, soil = pipe.mask(img_field)
    
    img_crop = pipe.createEmpty(y_sz, x_sz, n_band, datatype)
    
    for band in range(n_band):
        img_crop[:, :, band] = img_elc[:, :, band]*CC  
    
    img_soil = pipe.createEmpty(y_sz, x_sz, n_band, datatype)
    for band in range(n_band): 
        img_soil[:, :, band] = img_elc[:, :, band]*soil  
        
    #VIs
    NDRE = pipe.NVI(img_crop[:, :, 4],img_crop[:, :, 3], 0, 1)
    NDVI = pipe.NVI(img_crop[:, :, 4],img_crop[:, :, 2], 0, 1)
    GNDVI = pipe.NVI(img_crop[:, :, 4],img_crop[:, :, 1], 0, 1)
    BNDVI = pipe.NVI(img_crop[:, :, 4],img_crop[:, :, 0], 0, 1)

    ERVI = pipe.NVI(img_crop[:, :, 3],img_crop[:, :, 2], 0, 1)
    EGVI = pipe.NVI(img_crop[:, :, 3],img_crop[:, :, 1], 0, 1)
    EBVI = pipe.NVI(img_crop[:, :, 3],img_crop[:, :, 0], 0, 1)

    GRVI = pipe.NVI(img_crop[:, :, 1],img_crop[:, :, 2], 0, 1)
    GBVI = pipe.NVI(img_crop[:, :, 1],img_crop[:, :, 0], 0, 1)
    
    #Soil-based VIs
    WDVI, PVI, MSAVI2 = pipe.soilVIs(img_soil[:, :, 2],img_soil[:, :, 4],img_crop[:, :, 2],img_crop[:, :, 4], soil)
    
    #WIs
    NDWI = pipe.NVI(img_soil[:, :, 2],img_soil[:, :, 4], -1, 1)
    
    #VI Plot Extraction
    Indices = [NDRE,NDVI,GNDVI,BNDVI,ERVI,EGVI,EBVI,GRVI,GBVI,WDVI, PVI, MSAVI2]
    Names = ["NDRE","NDVI","GNDVI","BNDVI","ERVI","EGVI","EBVI","GRVI","GBVI","WDVI", "PVI", "MSAVI2"]
    
    Plot_ds = pipe.rasterize(plotFiles[i], y_sz, x_sz, ext, proj, ['ALL_TOUCHED=FALSE', 'ATTRIBUTE=ID'])
    roiPlot = Plot_ds.GetRasterBand(1).ReadAsArray().astype(np.uint32)
    Plotdata = gpd.read_file(plotFiles[i]).sort_values(by=['ID'])
    
    local_stats = pipe.Stats(roiPlot, Indices, Names) 
    local_stats['SEASON'] = season
    local_stats['DATASET'] = dataset
    join = Plotdata[["ID","TYPE","CLASS","SCORE"]].merge(local_stats, on='ID', how='inner')
    
    NDSI_stats=NDSI_stats.append(join, ignore_index=True)
    
    #WI Plot Extraction
    Indices = [NDWI]
    Names = ["NDWI"]
    
    local_stats = pipe.Stats (roiField, Indices, Names) 
    local_stats['SEASON'] = season
    local_stats['DATASET'] = dataset
    
    NDWI_stats=NDWI_stats.append(local_stats, ignore_index=True)
    
NDSI_stats = NDSI_stats.dropna() 
NDWI_stats = NDWI_stats.dropna() 

#Save the results
NDSI_stats.to_csv('NDSI.csv', index = True)
NDWI_stats.to_csv('NDWI.csv', index = True)

0it [00:00, ?it/s]

Processing dataset:2017B-1


1it [01:05, 65.85s/it]

Processing dataset:2017B-2


2it [02:08, 64.96s/it]

...

10it [13:28, 93.39s/it]

Processing dataset:2019B-2


11it [15:27, 101.20s/it]

Processing dataset:2019B-3


12it [17:15, 86.32s/it] 
